In [ ]:
import numpy as np
import pandas as pd
import json

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

El dataset cuenta con información sobre los potenciales clientes. Estas personas solicitaron una hipoteca, pero no se muestra si la venta llego al proceso final.

In [ ]:
df = pd.read_csv('data/dataset_v1.csv')
df.head(2)

In [ ]:
df.info()

In [ ]:
mask_solo_un_propietario = df['User 2 Birthdate'].isnull() != True
mask_solo_un_propietario2 = df['User 2 Salary'] < 1 
df[mask_solo_un_propietario & mask_solo_un_propietario2].head(2)


El dataset no presenta datos faltantes
Cuando solo hay un usuario, los datos de user2 se presentan como: 
- Fecha de nacimiento: NaN - 2779
- Trabajo: 0
- Salario: 0 - 2924

La diferencia se encuentra en personas que piden salario juntos, pero la segunda personas está en el paro

In [ ]:
# Tenemos más hipotecas individuales o compartidas?


Cargamos los datasets de registros y solicitudes de hipoteca

In [ ]:
df_registros = pd.read_csv('data/registros.csv')
df_solicitudes = pd.read_csv('data/solicitudes_de_hipoteca.csv')

In [ ]:
df_registros.head(2)

In [ ]:
df_registros_clean = df_registros.loc[:, ['user', 'id']]
df_registros_clean.columns = ['user', 'id_user']
df_registros_clean.head(2)

In [ ]:
df_solicitudes.head(2)

In [ ]:
df_solicitudes_clean = df_solicitudes.loc[:, ['id', 'user', 'mortgage_data']]
df_solicitudes_clean.columns = ['id_solicitud', 'user', 'mortgage_data']
df_solicitudes_clean.head(2)

In [ ]:
prueba = json.loads(df_solicitudes_clean['mortgage_data'][2])
prueba.keys()

In [ ]:
cantidad_hipo = prueba['data']['amount']
debt_ratio = prueba['data']['debtRatio']
total_income = prueba['revenue']['monthly'] # per month
date_solicitud = prueba['form_init_date']
pago_inicial = prueba['mortgage_initial']
mortgage_rate = prueba['data']['mortgage_rate']

pago_inicial

In [ ]:
#data_solicitudes =json.loads(df_solicitudes_clean['mortgage_data'])
amount = []
debt_ratio = []
total_income = []
date = []
mortgage_inital = []
mortgage_rate = []

for i in range(df_solicitudes.shape[0]):
    solicitud = json.loads(df_solicitudes_clean['mortgage_data'][i])
    amount.append(solicitud.get('data', {}).get('amount', None))
    total_income.append(solicitud.get('revenue',{}).get('monthly', None))
    debt_ratio.append(solicitud.get('data',{}).get('debtRatio', None))
    date.append(solicitud.get('form_init_date', None))
    mortgage_inital.append(solicitud.get('mortgage_initial', None))
    mortgage_rate.append(solicitud.get('data',{}).get('mortgage_rate',None))

In [ ]:
print(np.shape(amount),
np.shape(debt_ratio),
np.shape(total_income),
np.shape(date),
np.shape(mortgage_inital),
np.shape(mortgage_rate))

In [ ]:

df_info_solicitud = pd.DataFrame({
    'amount': amount,
    'debt_ratio': debt_ratio,
    'total_income': total_income,
    'date': date,
    'mortgage_inital': mortgage_inital,
    'mortgage_rate': mortgage_rate
})

In [ ]:
df_solicitudes_clean_2 = pd.concat([df_solicitudes_clean, df_info_solicitud], axis=1)
df_solicitudes_clean_2 = df_solicitudes_clean_2.drop('mortgage_data',axis=1)

In [ ]:
df_solicitudes_clean_2.head(2)

In [ ]:
df_registros_y_solicitudes = pd.merge(left=df_registros_clean, right=df_solicitudes_clean_2, left_on='user', right_on='user', how='inner')
df_registros_y_solicitudes.head(2)

In [ ]:
df_registros_y_solicitudes.info()

In [ ]:
df_ofertas = pd.read_csv('data/ofertas.csv')
df_ofertas_aceptadas = pd.read_csv('data/ofertas_aceptadas.csv')

In [ ]:
df_ofertas.head(2)

In [ ]:
df_ofertas_clean = df_ofertas.loc[:, ['id', 'mortgage_id', 'status']]
df_ofertas_clean.columns = ['id_oferta', 'mortgage_id', 'status_oferta']
df_ofertas_clean.head(2)

In [ ]:
df_ofertas_aceptadas.head(20)
df_ofertas_aceptadas.info()

In [ ]:
df_ofertas_aceptadas_clean = df_ofertas_aceptadas.loc[:, ['id', 'bid_id', 'closed_at']]
df_ofertas_aceptadas_clean.columns = ['id_oferta_aceptada', 'bid_id', 'closed_at']
df_ofertas_aceptadas_clean.head(2)

In [ ]:
df_ofertas_merged = pd.merge(left=df_ofertas_clean, right=df_ofertas_aceptadas_clean, left_on='id_oferta', right_on='bid_id')
df_ofertas_merged.head()

In [ ]:
df_ofertas_merged2 = df_ofertas_merged.drop('bid_id', axis=1)
df_ofertas_merged2.head(2)

In [ ]:
df_registros_y_solicitudes.head(2)

In [ ]:
print(f'Shape df_registros_y_solicitudes: {df_registros_y_solicitudes.shape}, \n shape df_ofertas_merged_2: {df_ofertas_merged2.shape}')

In [ ]:
df_merged = pd.merge(left=df_registros_y_solicitudes, right=df_ofertas_merged2, left_on='id_solicitud', right_on='mortgage_id', how='outer')
df_merged = df_merged.drop('mortgage_id', axis=1)
df_merged.head(10)

In [ ]:
df_merged.info()

In [ ]:
df_merged['total_income'] = pd.to_numeric(df_merged['total_income'], errors='coerce')
df_merged.info()

In [ ]:
df_merged.head(2)

In [ ]:
# Convertimos los tipos
df_merged['amount'] = df_merged['amount'].replace('', np.nan)
df_merged['amount'] = df_merged['amount'].astype(float)
df_merged['date'] = pd.to_datetime(df_merged['date'])
df_merged['mortgage_inital'] = df_merged['mortgage_inital'].replace('', np.nan)
df_merged['mortgage_inital'] = df_merged['mortgage_inital'].astype(float)
df_merged['closed_at'] = pd.to_datetime(df_merged['closed_at'], errors='coerce')
df_merged['closed_at'].apply(type).value_counts()

df = df_merged.copy()
df.info()


In [ ]:
def remove_outliers_iqr(df, column, iqr_multiplier=1.5):
    """
    Removes outliers from a specified column in a DataFrame using the IQR method.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column (str): The name of the column to remove outliers from.
        iqr_multiplier (float): The multiplier for the IQR to define the outlier fences.
                                  Commonly 1.5 for "mild" outliers, 3.0 for "extreme" outliers.

    Returns:
        pd.DataFrame: A new DataFrame with outliers removed.
    """
    if column not in df.columns:
        raise ValueError(f"Column '{column}' not found in the DataFrame.")

    # Calculate Q1, Q3, and IQR, ignoring NaN values
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    # Define outlier fences
    lower_bound = Q1 - iqr_multiplier * IQR
    upper_bound = Q3 + iqr_multiplier * IQR

    # Filter out outliers. Keep rows where the value is within the bounds OR is NaN.
    # We explicitly include NaN values so they are not treated as outliers and are preserved
    # if you want to handle them separately later.
    filtered_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound) | (df[column].isna())]

    # Optional: Print how many outliers were removed
    num_outliers = len(df) - len(filtered_df)
    print(f"Removed {num_outliers} outliers from '{column}' using IQR method (multiplier={iqr_multiplier}).")

    return filtered_df

df_clean = df.copy()
df_clean = remove_outliers_iqr(df_clean, 'amount', iqr_multiplier=1.5).copy()
df_clean = remove_outliers_iqr(df_clean, 'total_income', iqr_multiplier=1.5).copy() # Apply sequentially
df_clean = remove_outliers_iqr(df_clean, 'debt_ratio', iqr_multiplier=1.5).copy()

df = df_clean.copy()


# You can also chain the operations if you prefer
# df_clean_chained = df.pipe(remove_outliers_iqr, 'amount').pipe(remove_outliers_iqr, 'total_income').pipe(remove_outliers_iqr, 'debt_ratio')
# print("\nCleaned DataFrame (chained operations):")
# print(df_clean_chained)

In [ ]:
df.info()

In [ ]:
interesting_columns = ['amount', 'total_income', 'debt_ratio']
mask = df['closed_at'].notna()
mask_vendido = mask
fig, ax = plt.subplots(len(interesting_columns), 2)
for i in range(len(interesting_columns)):
    ax[i][0].hist(df[interesting_columns[i]],  bins=30, alpha=0.7)
    ax[i,0].hist(df.loc[mask, interesting_columns[i]],  bins=30, alpha=0.7, color='orange')
    ax[i,1].hist(df.loc[mask, interesting_columns[i]],  bins=30, alpha=0.7, color='orange')
    ax[i, 0].set_title(interesting_columns[i])
    ax[i, 1].set_title(interesting_columns[i] + ' exitosos')
    print(f'{interesting_columns[i]} mean: {df[interesting_columns[i]].mean()}, exitosos: {df.loc[mask, interesting_columns[i]].mean()}, diferencia: {df[interesting_columns[i]].mean() - df.loc[mask, interesting_columns[i]].mean()}')

plt.tight_layout()

In [ ]:
df_dates = df.loc[mask_vendido, ['date', 'closed_at']].copy()
df_dates.info()

In [ ]:
df_dates['closed_at'] = pd.to_datetime(df_dates['closed_at'], errors='coerce')
df_dates.info()
df_dates['duration'] = (df_dates['closed_at'] - df_dates['date']).dt.days
df_duration = remove_outliers_iqr(df_dates, 'duration', iqr_multiplier=1.5).copy()

In [ ]:
plt.hist(x=df_dates['duration'], bins=30)
print(f"Mean days: {df_dates['duration'].mean()}, moda days: {df_dates['duration'].mode()}")

In [ ]:
plt.hist(x=df_dates['duration'], bins=30)
plt.xlim(0,200)
print(f"Mean days: {df_dates['duration'].mean()}, moda days: {df_dates['duration'].mode()}")